In [16]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [5]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [8]:
data = spark.read.csv('/home/lalit/Desktop/spark-2.4.5-bin-hadoop2.7/python/PySpark/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv',inferSchema=True,header=True)

In [9]:
data.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [11]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [14]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [17]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [18]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership']
                            ,outputCol='features')

In [19]:
output = assembler.transform(data)

In [20]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [21]:
output.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [22]:
final_data = output.select(['features','Yearly Amount Spent'])

In [23]:
final_data.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows



In [24]:
train_data,test_data = final_data.randomSplit([0.8,0.2])

In [25]:
train_data.count()

398

In [26]:
test_data.count()

102

In [27]:
lr = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent', predictionCol='prediction')

In [28]:
model_lr = lr.fit(train_data)

In [29]:
test_results = model_lr.evaluate(test_data)

In [33]:
test_results.residuals.show(5)

+------------------+
|         residuals|
+------------------+
|11.531174513887436|
|-2.781398538264966|
|3.7180242495115863|
| 3.937135796659163|
| -4.55881719686414|
+------------------+
only showing top 5 rows



In [34]:
test_results.rootMeanSquaredError

10.747742659557339

In [35]:
test_results.r2

0.9837873755336257

In [37]:
unlabeled_data = test_data.select('features')

In [38]:
predictions = model_lr.transform(unlabeled_data)

In [39]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[31.1695067987115...| 415.8253562884054|
|[31.2681042107507...| 426.2519317120889|
|[31.3091926408918...|429.00269359042204|
|[31.3662121671876...|426.65174675982576|
|[31.5171218025062...|280.47723784724985|
|[31.6098395733896...|426.43412087684146|
|[31.7656188210424...| 501.1961931153503|
|[31.8293464559211...|  383.828384544998|
|[31.8648325480987...| 449.3033049892929|
|[31.9120759292006...|388.69514581402996|
|[31.9673209478824...|450.71044531069447|
|[32.0305497162129...|  589.060145294264|
|[32.0444861274404...|445.94179518916303|
|[32.0775900443291...|403.77701668867803|
|[32.0883806304482...| 517.5524676934408|
|[32.1151190660142...| 342.4000179323739|
|[32.1924988277864...| 619.3153585270118|
|[32.2301491159331...| 514.8296466742204|
|[32.2718482808749...|506.75365901083137|
|[32.2745938303391...| 523.3704702111006|
+--------------------+------------